## **Student's Autograder**
This is a project to calculate quiz and term scores from q&a pdf and response pdf.

Whats left: 

* Editing pdf


### Importing the required libraries 

In [37]:
import os
import pandas as pd
import numpy as np
import tabula
from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, EncodedStreamObject,ArrayObject

In [38]:
#current directory for reading files later.
import os
os.getcwd()

'/home/hem436/Documents/TDS/Week 2'

### Reading pdfs

In [39]:
# Enter response pdf name
resp_pdf = "POD22S1C21530041.pdf"

#POD22S1C11500062.pdf

#Enter corresponding question and answer key pdf name
qa_pdf = "IIT M DIPLOMA FN2 EXAM ETD2.pdf"

#IIT M DAD DIPLOMA AN2 EXAM QPD1.pdf


In [40]:
filename=qa_pdf

Students response for exam dataframe

In [41]:
rpdf_df=PdfFileReader(resp_pdf)

In [42]:
#Updated response pdf reading method from tabula to raw 
myans=[]
for i in range(rpdf_df.getNumPages()):
    myans+=(rpdf_df.pages[i].extractText().split("\n"))

#### Variables initialization

In [43]:
resp_qid,my_answer=myans[11:13]
unanswered="Unanswered"
file_qid="Question Id"

In [44]:
myans=myans[myans.index(resp_qid):]
myans=pd.DataFrame(np.vstack((myans[2::2],myans[3::2])).T,columns=[myans[0],myans[1]])
myans[resp_qid]=myans[resp_qid].apply(int)
myans

,Question Id,Options Selected
0,640653451668,Unanswered
1,640653451670,Unanswered
2,640653451672,Unanswered
3,640653451673,Unanswered
4,640653451676,Unanswered
...,...,...
356,640653452050,Unanswered
357,640653452054,Unanswered
358,640653452042,Unanswered
359,640653452044,Unanswered


In [45]:
# with pd.option_context("display.max_rows",None):
#     print(df3)

### Useful functions

In [46]:
def gentextarr(content):
    """
    Returns an array of text block properties of pdf
    Parameter:
        content: string object of page content obtained by page.getContents().getData().decode() 
    """
    lines = content.splitlines()
    result = []
    in_text = False
    for line in lines:
        if line == "BT":
            in_text = True
            temp=[]

        elif line == "ET":
            in_text = False
            result.append(temp)
            temp=[]

        elif in_text:
            if line[-2:].lower()=="tj" or line[-2:].lower()=="tf" or line[-2:].lower()=="rg":
                temp.append(line[:])
    return result

def gen_cmap(font,pg):
    """
    Returns a character map dictionary of a pdf 'filename' page 'pg' and font 
    Parameter:
     font: font key name in meta properties of pdf.
     pg: int page number of pdf
    """
    # pdf = PdfFileReader(filename)
    # page = pdf.getPage(pg)
    page=pg
    temp=False
    cmap={}
    for i in (page.getObject()['/Resources']['/Font'][font]['/ToUnicode'].getObject().getData().decode().split('\n')):
        if "endbfrange" in i:
            temp=False
        if temp:
            cmap[i[1:5]]=i[-5:-1]
        if "beginbfrange" in i:
            temp=True
    return cmap

def font_decoder(text,cmap):
    """
    Returns decoded word from pdf text hex code. 
    """

    count=0
    word=""
    temp=""
    for i in text:
        count+=1
        if count==4:
            temp+=i
            word+=chr(int(cmap.get(temp.lower(),temp.lower()),16))
            temp=""
            count=0
        else:
            temp+=i
    return word


## PDF Content

In [47]:
"""
This piece of code extracts question answer pdf text content as well as correct answer key.
"""
pdf = PdfFileReader(filename)
sum=0
df=[[file_qid,"Correct Marks","Answers"]]
m_bool=False
marks=""
ans_bool=False
ans=""
row=[]
temp=[]
s_head=[]
temp_head=""
ishead=False
for p in range(0,pdf.getNumPages()):
    content=pdf.pages[p].getContents()
    #a different type content
    if type(content)==ArrayObject:
        m=""
        for n in content:
            m+=n.getObject().getData().decode()
        content=m
    elif type(content)==EncodedStreamObject or type(content)==DecodedStreamObject:
        content=content.getData().decode()
    for i in gentextarr(content):
        text=""
        ans_bool=False
        for j in i:
            # getting character map for a font type
            if j[-2:].lower()=="tf":
                cmap=gen_cmap(j.split()[0],pdf.pages[p])
                #checking for subject headings with font size =18
                if j.split()[1]=="18":
                    ishead=True
            # checking green color as ind of correctness        
            if j=="0 0.50196 0 rg" and len(row)>1:
                ans_bool=True
    
            elif j[-2:].lower()=="tj":
                text=font_decoder(j[1:-3],cmap)
                if ishead:
                    ishead=False
                    temp_head=text
                
                if (not m_bool) and (not ans_bool):
                    # checking for question id
                    if str(file_qid) in text:
                        if len(temp)>0:
                            row.append(temp)
                            df.append(row)
                        row=[]
                        temp=[]
                        if text.split()[7].isnumeric():
                            s_head.append(temp_head)
                            row=[text.split()[7]]
                            m_bool=True
                    
                # extracting marks
                elif m_bool:
                    if "Correct Marks" in text:
                        # appending to heading list if marks is 0
                        # if text.split()[-1]=="0":
                        #     s_head.append(temp_head)
                        if len(row)==1:
                            row.append(text.split()[3])
                        m_bool=False
                        
                #extracting answer key
                elif ans_bool:
                    if text!=" ":
                        if len(text)>=13 and text[:13].isnumeric():
                            temp.append(text[:13])
                        elif len(text.split())==3 and "to" in text.split()[1]:
                            t=text.split()
                            if not(t[0].isalpha() or t[-1].isalpha()):
                                temp=[float(t[0]),float(t[-1])]
                        elif len(temp)==0:
                            temp.append(text)
if len(temp)>=1:
    # s_head.append(temp_head)
    row.append(temp)
    df.append(row)

In [48]:
df2=pd.DataFrame(df[1:],columns=df[0])
df2[file_qid]=df2[file_qid].apply(int)
df2["Subjects"]=np.array(s_head[-df2.shape[0]:])
df2=df2.sort_values([file_qid])
# df2[df2["Subjects"]=="MLT"]["Correct Marks"].apply(float).sum()

In [49]:
def evaluate(df2):
    
    #Multi option
    df2['Score'] = df2.apply(lambda x: (len(set(x["My answer"].split(","))\
                                            .intersection(set(x["Answers"])))/len(x["Answers"]))*float(x['Correct Marks'])\
                             if type(x["Answers"][0])==str and len(x["Answers"][0])==13 and\
                                len(set(x["My answer"].split(",")))<=len(set(x["Answers"])) and\
                                set(x["My answer"].split(",")).issubset(set(x["Answers"]))\
                             else x["Score"], axis=1)
    #integer ranged
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
                             if len(x["My answer"])<13 and len(x["My answer"])>0 and\
                                len(x['Answers'])>=1 and\
                                float(x['My answer']) >= float(x["Answers"][0]) and\
                                float(x['My answer']) <= float(x["Answers"][-1])\
                             else x["Score"], axis=1)
    return df2

## Final score

### Method 1: using index of 0 correct marks

In [50]:
# s_index=df3[df3["Correct Marks"]=="0"].index
# s_index

In [51]:
# subjects={}
# for i in range(len(s_index)):
#     if i==len(s_index)-1:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:]
#     else:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:s_index[i+1]]
# subjects.keys()

In [52]:
# reports=[]
# for i in subjects.values():
#     print(i["Correct Marks"].apply(float).values)
#     reports.append([i["Subjects"].unique()[0],i["Correct Marks"].apply(float).values.sum(),i["Score"].sum()])
# for i in reports:
#     print(i[0],":",i[2]/i[1]*100,"%")

In [53]:
# with pd.option_context("display.max_rows",None):
#     display(df)

### Method 2: Using groupby function

In [55]:
df3=df2.set_index(file_qid).combine_first(myans.set_index(resp_qid)).reset_index().replace([unanswered,np.NAN],"")

In [56]:
df3.columns=["Question Id","Answers","Correct Marks","My answer","Subjects" ]
df3["Score"]=np.zeros(df3.shape[0])
df3=evaluate(df3)

In [57]:
df3["Correct Marks"]=df3["Correct Marks"].apply(float)

In [58]:
with pd.option_context("display.max_rows",None):
    display(df3[df3["Subjects"].isin(df3[df3["My answer"]!=""]["Subjects"].unique())])


,Question Id,Answers,Correct Marks,My answer,Subjects,Score
24,640653451692,[6406531503179],0.0,6406531503179,MLT,0.0
25,640653451693,[35],5.0,32.2,MLT,0.0
26,640653451695,[15],3.0,15,MLT,3.0
27,640653451696,[6406531503185],3.0,6406531503186,MLT,0.0
28,640653451697,[6406531503187],5.0,6406531503187,MLT,5.0
29,640653451699,[5],4.0,5,MLT,4.0
30,640653451700,"[6406531503193, 6406531503194]",4.0,"6406531503193,6406531503194",MLT,4.0
31,640653451701,[6406531503198],4.0,6406531503198,MLT,4.0
32,640653451703,[3],1.5,3,MLT,1.5
33,640653451704,[1],1.5,1,MLT,1.5


In [59]:
report2=pd.DataFrame(df3.groupby("Subjects")[["Correct Marks","Score"]].sum())

In [60]:
report2["Percentage"]=(report2["Score"]*100/report2["Correct Marks"]).round(2)

In [61]:
report2.columns=["Total Marks","Scored","Percentage"]
headers = {
    'selector': 'thead',
    'props': 'background-color: #0dcaf0; color: white;'
}
report2.style.applymap(lambda v: 'color:red;' if (v == 0) else 'color:green',subset=["Percentage","Scored"])\
    .set_table_styles([headers])\
    .format({"Total Marks":'{:.2f}',"Scored":'{:.2f}',"Percentage":'{0:,.2f}%'})

,Total Marks,Scored,Percentage
Subjects,,,
AppDev1,100.00,0.00,0.00%
AppDev2,100.00,0.00,0.00%
BDM,40.00,0.00,0.00%
Business Analytics,50.00,0.00,0.00%
DBMS,50.00,0.00,0.00%
Java,100.00,0.00,0.00%
MLF,50.00,0.00,0.00%
MLP,100.00,0.00,0.00%
MLT,100.00,82.50,82.50%


In [62]:
report2[report2["Percentage"]>0]

,Total Marks,Scored,Percentage
Subjects,,,
MLT,100.0,82.5,82.5
TDS,50.0,41.0,82.0


### Exporting dataframe as csv

In [ ]:
df3[df3["Subjects"].isin(report2[report2["Percentage"]>0].index)].to_csv("summary.csv",index=False)
report2[report2["Percentage"]>0].to_csv("report_card.csv")